In [1]:
import pandas
import random
import numpy as np
import math
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import sys
import time
import argparse
import pprint


### Collect data

place your data at the same floder with the github project

In [2]:
# input path of file
# return data extract from pickle
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# examples of how to get data from the file
# data stores a collection of images in flat mode
# label stores the labels corresponding to each image
DIR = "../../cifar-10-batches-py"
FILE_LIST = ["data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4", "data_batch_5", "test_batch"]

rawdata = unpickle(DIR + "/" + FILE_LIST[0])

data = rawdata[b'data']
labels = rawdata[b'labels']
batch_label = rawdata[b'batch_label']
filenames = rawdata[b'filenames']


### transform the raw data into image
below gives an example of how to extract the first image from the data


change index to get more

In [3]:
# input 2D data
# return BGR image
def flatToImg(flat):
    img = np.reshape(flat, (3, 32, 32))
    img = np.transpose(img, (1, 2, 0))
    return img

# read and show image
flat = data[0]
# img = flatToImg(flat)
# print(img.shape)
# plt.imshow(img)
# plt.show()

### extract secret from image and decode

In [4]:
# input: image
# return bits of image in one-dimention like
def getSecret(image):
    return np.unpackbits(image.flatten())
  
# image = np.array([[1, 2], [3, 4]], dtype=np.uint8)
# getSecret(image)

In [5]:
# input: secret data s(binary), bool value
# return decimal data, if ture return a BGR image, else 2D value
def decode(s, toImg=False):
    if not toImg:
        return np.packbits(s)
    else:
        return flatToImg(np.packbits(s))

    
# s = getSecret(image)
# img = decode(s, True)
# plt.imshow(img)
# plt.show()

## synthesizing malicious data

In [6]:
def GenData(d1, d2):
    randVal = random.randint(0,255)
    image = np.zeros([d1, d2])
    idx1 = random.randint(0, d1-1)
    idx2 = random.randint(0, d2-1)
    image[idx1, idx2] = randVal
    return image

# test
x = GenData(5, 5)
print(x)
# print(x[:2].flatten)

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [14.  0.  0.  0.  0.]]


In [7]:
def bitToLabel(bits, label_num):
    x = math.ceil(len(bits)/label_num)
    label = [0]*x
    for i in range(x):
        label[i] = int(bits[i*label_num:(i+1)*label_num], 2)
    return label
        
bits = "0011010011"
bitToLabel(bits, 3)
    

[1, 5, 1, 1]

In [8]:
!pip install keras

In [11]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 3
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[:100]
y_train = y_train[:100]
x_test = x_test[:100]
y_test = y_test[:100]
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(320, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(320, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (100, 32, 32, 3)
100 train samples
100 test samples
Using real-time data augmentation.
Epoch 1/3
4/4 [==============================] - 8s 2s/step - loss: 2.3148 - accuracy: 0.1000 - val_loss: 2.3088 - val_accuracy: 0.0600
Epoch 2/3
4/4 [==============================] - 8s 2s/step - loss: 2.2993 - accuracy: 0.1300 - val_loss: 2.3209 - val_accuracy: 0.0600
Epoch 3/3
4/4 [==============================] - 7s 2s/step - loss: 2.2410 - accuracy: 0.1800 - val_loss: 2.3619 - val_accuracy: 0.0600
Saved trained model at C:\Users\ruoya\Documents\5271\5271_Final_project\blackBox\saved_models\keras_cifar10_trained_model.h5 
100/100 [==============================] - 2s 17ms/step
Test loss: 2.3618683338165285
Test accuracy: 0.05999999865889549
